In [ ]:
!pip3 install tensorflow-gpu

In [2]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.python.client import device_lib

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [4]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14511637391672205993
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14425078642202964536
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6762313491399021946
physical_device_desc: "device: XLA_CPU device"
]


In [5]:
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [6]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

print( tf.VERSION )

import json
import gzip


filename = 'qa_Electronics.json.gz'  # Sample file.
questions=[]
answers=[]
i=0
with gzip.open(filename , 'rb') as gzip_file:
    for line in gzip_file:  # Read one line.
        i+=1
        l=eval(line)
        questions.append(l['question'])
        answers.append( '<START> ' + l['answer'] + ' <END>' )
        if i==10000:
            break
       
        

tokenizer = preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,)
tokenizer.fit_on_texts( questions + answers )
count_thres = 3
low_count_words = [w for w,c in tokenizer.word_counts.items() if c < count_thres]

for w in low_count_words:
    del tokenizer.word_index[w]
    del tokenizer.word_docs[w]
    del tokenizer.word_counts[w]

VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))  

1.14.0
VOCAB SIZE : 7154


In [10]:
import os
embeddings_index = {}
f = open(os.path.join('glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
max_features = 20000
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 200

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

Found 7153 unique tokens.
7154


In [ ]:
questions[1]

In [17]:
tokenized_questions = tokenizer.texts_to_sequences( questions )

In [18]:
len(tokenized_questions)

50000

In [13]:
from tensorflow.keras import preprocessing , utils

import os
import yaml
import numpy as np
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = 20
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = 30
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(10000, 20) 20
(10000, 30) 30
(10000, 30, 7154)


In [14]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.initializers import Constant
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 ,embeddings_initializer=Constant(embedding_matrix), mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 ,embeddings_initializer=Constant(embedding_matrix) ,mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    1430800     input_3[0][0]                    
________________________________________________________________________________

In [15]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=30 ) 

Epoch 1/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 3.8744
Epoch 2/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 2.6825
Epoch 3/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 2.0596
Epoch 4/30
10000/10000 [==============================] - 105s 10ms/sample - loss: 1.7650
Epoch 5/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 1.5944
Epoch 6/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 1.4788
Epoch 7/30
10000/10000 [==============================] - 105s 10ms/sample - loss: 1.3973
Epoch 8/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 1.3312
Epoch 9/30
10000/10000 [==============================] - 107s 11ms/sample - loss: 1.2684
Epoch 10/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 1.2169
Epoch 11/30
10000/10000 [==============================] - 106s 11ms/sample - loss: 1.1711
Epoch 12

In [16]:
model.save( 'model-sequence-embedding.h5' ) 

In [17]:
import tensorflow as tf 
model = tf.keras.models.load_model('model-sequence-embedding.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [19]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [22]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question : who is this
 i don't know the answer is what i bought this in my question i use mine for my new one end
Enter question : what is battery life
 very good and getting on one of the unit but you get one with the tripod for the tripod mount end
Enter question : does it last long
 i don't know the answer is but i bought this for my case end
Enter question : how is the battery life
 the it's a 3 5 feet long end
Enter question : will this fit my case
 yes it will end
Enter question : who are you
 yes they are just better end
Enter question : what will it be
 it might be a i would think that would work in my opinion end
Enter question : how may days will it last
 i have to say it on a small at that it was made in my opinion end
Enter question : what are opinions
 i purchased mine for 8 8 8 no longer available in end
Enter question : how many days
 hi 10 1 4 x 1 4 x 1 4 x 1 1 end
